In [158]:
import pandas as pd
import datetime
from sqlalchemy import create_engine

In [159]:
# 連接 sql 引擎
host='database-1.cyn7ldoposru.us-east-1.rds.amazonaws.com'
port='5432'
user='Yu'
password='m#WA12J#'
database="JQC_Revenue1"

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
        user, password, host, port, database), echo=True)

In [212]:
otc_year = datetime.datetime.today().year - 1911
otc_date = str(otc_year) + '/' + datetime.datetime.today().strftime("%Y/%m/%d")[5:]
tse_date = ''.join(datetime.datetime.today().strftime("%Y/%m/%d").split('/'))

tse_url = 'https://www.twse.com.tw/rwd/zh/afterTrading/MI_INDEX?date=' + tse_date + '&type=ALLBUT0999&response=html'
otc_url = 'https://www.tpex.org.tw/web/stock/aftertrading/otc_quotes_no1430/stk_wn1430_result.php?l=zh-tw&o=htm&d=' + otc_date + '&se=EW&s=0,asc,0'
tse = pd.read_html(tse_url)[-1]
otc = pd.read_html(otc_url)[0]
tse.columns = tse.columns.droplevel(level=0)
otc.columns = otc.columns.droplevel(level=0)
otc = otc[otc['次日漲停價'].isna() == False]

otc['date'] = datetime.datetime.today().strftime("%Y/%m/%d")
tse['date'] = datetime.datetime.today().strftime("%Y/%m/%d")

otc = otc.loc[:, ['代號','名稱','date','開盤','最高','最低','收盤','成交股數']]
otc = otc.set_axis(['st_code', 'st_name', 'date', 'open', 'high', 'low', 'close', 'volume'], axis=1)
tse = tse.loc[:, ['證券代號','證券名稱','date','開盤價','最高價','最低價','收盤價','成交股數']]
tse = tse.set_axis(['st_code', 'st_name', 'date', 'open', 'high', 'low', 'close', 'volume'], axis=1)

# Update to sql
tse.reset_index(drop=False).to_sql('daily_trading', engine, if_exists='append')
otc.reset_index(drop=False).to_sql('daily_trading', engine, if_exists='append')

2023-03-27 15:42:38,998 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2023-03-27 15:42:39,003 INFO sqlalchemy.engine.Engine [cached since 5.388e+04s ago] {'name': 'daily_trading'}


OperationalError: (psycopg2.OperationalError) SSL SYSCALL error: Operation timed out

[SQL: select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s]
[parameters: {'name': 'daily_trading'}]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [211]:
pd.concat([tse, otc], axis=0)

,st_code,st_name,date,open,high,low,close,volume
0,0050,元大台灣50,2023/03/27,121.75,121.90,121.20,121.45,11001040
1,0051,元大中型100,2023/03/27,57.50,57.60,57.05,57.05,46477
2,0052,富邦科技,2023/03/27,109.90,109.90,109.10,109.30,157190
3,0053,元大電子,2023/03/27,60.80,61.20,60.80,61.20,6729
4,0055,元大MSCI金融,2023/03/27,21.83,21.88,21.72,21.84,339439
...,...,...,...,...,...,...,...,...
901,020025,統一亞洲半導體N,2023/03/27,6.95,6.97,6.95,6.97,9000
902,020026,兆豐上櫃ESG電菁N,2023/03/27,----,----,----,----,0
903,020027,元大上櫃ESG成長N,2023/03/27,4.63,4.67,4.63,4.64,222000
904,020033,統一恒生科期N,2023/03/27,3.84,3.85,3.78,3.82,88000
